In [137]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [162]:
# SETTINGS

learning_rate = 1e-2
torch.manual_seed(42)
device = 'cpu'
if torch.cuda.is_available():
	device = 'cuda'

print(device)

cpu


In [163]:
mnist_train = torchvision.datasets.MNIST('.', train=True, download=True)
mnist_test = torchvision.datasets.MNIST('.', train=False, download=True)

data_train = mnist_train.data.to(device).float()
data_test = mnist_test.data.to(device)

target_train = mnist_train.targets.to(device).float()
target_test = mnist_test.targets.to(device)

print(data_train.shape, data_test.shape)
print(data_train.dtype, data_test.dtype)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])
torch.float32 torch.uint8


In [164]:
class Model(nn.Module):
	def __init__(self):
		super().__init__()
		self.linear1 = nn.Linear(784, 800)
		self.linear2 = nn.Linear(800, 10)
		
	def forward(self, x, y=None):
		r = x.view(-1, 784)
		r = F.relu(self.linear1(r))
		r = self.linear2(r)
		if y is None:
			loss = None
		else:
			loss = F.cross_entropy(r, y)
		return r, loss

	def predict(self, x):
		logits, _ = self.forward(x)
		return logits.argmax()

model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [161]:
estimation_iters = 100
estimation_batch = 32

def get_batch(split):
	data = data_train if split == 'train' else data_test
	target = target_train if split == 'train' else data_test
	ix = torch.randint(0, len(data), (estimation_batch,))
	x = torch.stack([data[i] for i in ix])
	y = torch.stack([target[i] for i in ix])
	return x, y

@torch.no_grad()
def estimate_loss(split):
	losses = torch.zeros(estimation_iters)
	for i in range(estimation_iters):
		x, y = get_batch(split)
		logits, loss = model.forward(x, y)
		losses[i] = loss
	return losses.mean()

estimate_loss('train'), estimate_loss('test')

RuntimeError: expected scalar type Long but found Float

In [129]:
training_iters = 10000
training_batch = 32
eval_interval = 1000
for iter in range(max_iters):
	if (iter % eval_interval) == 0:
		print(f'iter {iter}: loss={estimate_loss()}')
	x = data_train.data[iter]
	y = data_train.targets[iter]
	logits, loss = model(x, y)
	optimizer.zero_grad()
	loss.backward()
	optimizer.step()


In [141]:
def predict_one(index):
	print(model.predict(data_train.data[index].to(device).float()))
	return images_train[index][0]

predict_one(0)

tensor(1)
